In [1]:
import vrep 
import sys
import time 
import skfuzzy as fuzz
import numpy as np
from tank import *

# define the universe

In [2]:
in_dist = np.arange(0.7, 5.1, 0.1)
in_velo = np.arange(0, 10.1, 0.1)
out_velo  = np.arange(0, 10.1, 0.1)

# Generate fuzzy membership functions

In [3]:
dist_lo = fuzz.trimf(in_dist, [0.7, 0.7, 3.0])
dist_md = fuzz.trimf(in_dist, [0.7, 3.0, 5.1])
dist_hi = fuzz.trimf(in_dist, [3.0, 5.1, 5.1])

velo_lo = fuzz.trimf(in_velo, [0, 0, 5])
velo_md = fuzz.trimf(in_velo, [0, 5, 10])
velo_hi = fuzz.trimf(in_velo, [5, 10, 10])

out_velo_lo = fuzz.trimf(out_velo, [0, 0, 5])
out_velo_md = fuzz.trimf(out_velo, [0, 5, 10])
out_velo_hi = fuzz.trimf(out_velo, [5, 10, 10])

# Mamdami rule

In [4]:
def fuzzy(velo, dist):
    # for distance
    dist_level_lo = fuzz.interp_membership(in_dist, dist_lo, dist)
    dist_level_md = fuzz.interp_membership(in_dist, dist_md, dist)
    dist_level_hi = fuzz.interp_membership(in_dist, dist_hi, dist)

    # for velocity
    velo_level_lo = fuzz.interp_membership(in_velo, velo_lo, velo)
    velo_level_md = fuzz.interp_membership(in_velo, velo_md, velo)
    velo_level_hi = fuzz.interp_membership(in_velo, velo_hi, velo)

    # rules for decelleration
    dec_rule1 = np.fmin(dist_level_md, out_velo_hi)
    dec_rule2 = np.fmin(dist_level_lo, out_velo_lo)
    dec_rule = np.fmax(dec_rule1, dec_rule2)

    # rules for no operation
    nop_rule1 = np.fmin(dist_level_md, out_velo_md)
    nop_rule2 = np.fmin(dist_level_hi, out_velo_hi)
    nop_rule = np.fmin(nop_rule1, nop_rule2)

    # rules for acceleration
    acc_rule1 = np.fmin(dist_level_md, velo_level_lo)
    acc_rule2 = np.fmin(dist_level_hi, out_velo_lo)
    acc_rule3 = np.fmin(dist_level_hi, out_velo_md)
    acc_rule = np.fmax(acc_rule1, acc_rule2, acc_rule3)

    #combine everything
    v_activation_dec = np.fmin(dec_rule, out_velo_lo)  
    v_activation_nop = np.fmin(nop_rule, out_velo_md)
    v_activation_acc = np.fmin(acc_rule, out_velo_hi)
    aggregated = np.fmax(v_activation_dec, np.fmax(v_activation_nop, v_activation_acc))

    return fuzz.defuzz(out_velo, aggregated, 'lom') #centroid

In [5]:
def fuzzy_forward(velo, dist):
    # for distance
    dist_level_lo = fuzz.interp_membership(in_dist, dist_lo, dist)
    dist_level_md = fuzz.interp_membership(in_dist, dist_md, dist)
    dist_level_hi = fuzz.interp_membership(in_dist, dist_hi, dist)

    # for velocity
    velo_level_lo = fuzz.interp_membership(in_velo, velo_lo, velo)
    velo_level_md = fuzz.interp_membership(in_velo, velo_md, velo)
    velo_level_hi = fuzz.interp_membership(in_velo, velo_hi, velo)


    # rules for decelleration
    dec_rule1 = np.fmin(dist_level_md, out_velo_hi)
    dec_rule2 = np.fmin(dist_level_lo, out_velo_lo)
    dec_rule = np.fmax(dec_rule1, dec_rule2)

    # rules for no operation
    nop_rule1 = np.fmin(dist_level_md, out_velo_md)
    nop_rule2 = np.fmin(dist_level_hi, out_velo_hi)
    nop_rule = np.fmin(nop_rule1, nop_rule2)

    # rules for acceleration
    acc_rule1 = np.fmin(dist_level_md, velo_level_lo)
    acc_rule2 = np.fmin(dist_level_hi, out_velo_lo)
    acc_rule3 = np.fmin(dist_level_hi, out_velo_md)
    acc_rule = np.fmax(acc_rule1, acc_rule2, acc_rule3)

    #combine everything
    v_activation_dec = np.fmin(dec_rule, out_velo_lo)  
    v_activation_nop = np.fmin(nop_rule, out_velo_md)
    v_activation_acc = np.fmin(acc_rule, out_velo_hi)
    aggregated = np.fmax(v_activation_dec, np.fmax(v_activation_nop, v_activation_acc))

    return fuzz.defuzz(out_velo, aggregated, 'lom') #centroid

The "lom" (or "Smallest Of Maximum") method is used to perform the defuzzification, which involves finding the centroid (center of mass) of the fuzzy set.

In [6]:
vrep.simxFinish(-1) # closes all opened connections, in case any prevoius wasnt finished
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # start a connection

if clientID!=-1:
    print ("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

#create instance of Tank
tank=Tank(clientID)

Connected to remote API server


In [7]:
proximity_sensors=["EN","ES","NE","NW","SE","SW","WN","WS"]

proximity_sensors_handles=[0]*len(proximity_sensors)

# get handle to proximity sensors
for i in range(len(proximity_sensors)):
    err_code,proximity_sensors_handles[i] = vrep.simxGetObjectHandle(clientID,"Proximity_sensor_"+proximity_sensors[i], vrep.simx_opmode_blocking)
    
#read and print values from proximity sensors
#first reading should be done with simx_opmode_streaming, further with simx_opmode_buffer parameter
for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_streaming)

# 1 forward

In [8]:
tank.forward(10)

#continue reading and printing values from proximity sensors
run = True
while run: # read values for 5 seconds
    for sensor_name, sensor_handle in zip(proximity_sensors, proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
        
        if(err_code == 0):

            reading = np.linalg.norm(detectedPoint)
            print(reading)
            velo = fuzzy(tank.rightvelocity, reading)


            if(sensor_name == 'EN' ):
                close = fuzz.interp_membership(in_dist, dist_lo, reading)
                mid = fuzz.interp_membership(in_dist, dist_md, reading)
                far = fuzz.interp_membership(in_dist, dist_hi, reading)

                if close > far:
                    print("Ending")
                    run = False
                
                velo = fuzzy(tank.rightvelocity, reading)
                tank.forward(velo)


    print()

1.1015112476871536
Ending
1.1015112476871536
1.4006667802025985
3.001979857630172
3.001979857630172
3.001979857630172
2.478786269325544
2.478786269325544



# right

In [60]:
tank.forward(10)

#continue reading and printing values from proximity sensors
run = True
while run: # read values for 5 seconds
    for sensor_name, sensor_handle in zip(proximity_sensors, proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
        
        if(err_code == 0):

            reading = np.linalg.norm(detectedPoint)
            print(reading)
            velo = fuzzy(tank.rightvelocity, reading)

            if velo <= 0.00001:
                print("Ending")
                run = False

            if(sensor_name == 'NE'):
                velo = fuzzy(tank.rightvelocity, reading)

            tank.turn_right(velo)

    print()

1.1129704476004525
1.1129704476004525
1.44825950963886
2.661275161781219
2.661275161781219
2.661275161781219
2.5130027198142093
2.5130027198142093

1.1129704476004525
1.1129704476004525
1.44825950963886
2.661275161781219
2.661275161781219
2.661275161781219
2.5130027198142093
2.5130027198142093

1.1129704476004525
1.1129704476004525
1.44825950963886
2.661275161781219
2.661275161781219
2.661275161781219
2.5130027198142093
2.5130027198142093

1.1129704476004525
1.1129704476004525
1.44825950963886
2.661275161781219
2.661275161781219
2.661275161781219
2.5130027198142093
2.5130027198142093

1.1129704476004525
1.1129704476004525
1.44825950963886
2.661275161781219
2.661275161781219
2.661275161781219
2.5130027198142093
2.5130027198142093

1.1129704476004525
1.1129704476004525
1.44825950963886
2.661275161781219
2.661275161781219
2.661275161781219
2.5130027198142093
2.5130027198142093

1.1129704476004525
1.1129704476004525
1.44825950963886
2.661275161781219
2.661275161781219
2.661275161781219
2.5

KeyboardInterrupt: 

 # forward

In [ ]:
tank.forward(10)

#continue reading and printing values from proximity sensors
run = True
while run: # read values for 5 seconds
    for sensor_name, sensor_handle in zip(proximity_sensors, proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
        
        if(err_code == 0):

            reading = np.linalg.norm(detectedPoint)
            print(reading)
            velo = fuzzy(tank.rightvelocity, reading)

            if velo <= 0.00001:
                print("Ending")
                run = False

            if(sensor_name == 'NW' ):
                velo = fuzzy(tank.rightvelocity, reading)

            tank.forward(velo)

    print()

# left

In [ ]:
tank.forward(10)

#continue reading and printing values from proximity sensors
run = True
while run: # read values for 5 seconds
    for sensor_name, sensor_handle in zip(proximity_sensors, proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
        
        if(err_code == 0):

            reading = np.linalg.norm(detectedPoint)
            print(reading)
            velo = fuzzy(tank.rightvelocity, reading)

            if velo <= 0.00001:
                print("Ending")
                run = False

            if(sensor_name == 'SW'):
                velo = fuzzy(tank.rightvelocity, reading)

            tank.turn_left(velo)

    print()